In [4]:
from peewee import *
from models import *


db = SqliteDatabase("./space_tourism.db")
db.connect()

True

list(cursor.execute('''\
SELECT \
    t.name AS tour_name,\
    GROUP_CONCAT(DISTINCT t.date) AS tour_date,\
    AVG(tt.price) AS avg_ticket_price,\
    COUNT(tt.client_id) AS client_count\
FROM tours t\
JOIN tour_tickets tt ON t.id = tt.tour_id\
WHERE t.date > '2024-01-01'\
GROUP BY t.name\
HAVING COUNT(tt.client_id) > 2\
ORDER BY avg_ticket_price DESC;          \
                    \
'''))

In [12]:
query = (Tours
         .select(Tours.name.alias("tour_name"), fn.GROUP_CONCAT(Tours.date.distinct()).alias("tour_data"),
                 fn.AVG(TourTickets.price).alias("avg_ticket_price"),
                 fn.COUNT(TourTickets.client_id).alias("client_count"))
         .join(TourTickets)
         .where(Tours.date > "2024-01-01")
         .group_by(Tours.name)
         .having(fn.COUNT(TourTickets.client) > 2)
         .order_by(SQL('tour_name').desc())

         )



l = [[i.tour_name, i.tour_data, i.avg_ticket_price, i.client_count] for i in query]
l

[['Mars Expedition', '2025-07-26,2025-05-05', 14519.333333333334, 3],
 ['Jupiter Adventure', '2025-05-20,2025-06-25', 13786.555555555555, 9],
 ['ISS Experience',
  '2025-07-31,2025-07-06,2025-07-20,2025-06-12',
  13182.125,
  16]]

list(cursor.execute('''\
SELECT \
    c.name || ' ' || c.surname AS client_name,\
    r.text AS review_text,\
    r.date AS review_date,\
    t.name AS tour_name,\
    AVG(tt.price) AS avg_tour_price\
FROM clients c\
JOIN reviews r ON c.id = r.client_id\
JOIN tours t ON r.tour_id = t.id\
JOIN tour_tickets tt ON t.id = tt.tour_id\
GROUP BY c.id\
HAVING avg_tour_price > (SELECT AVG(price) FROM tour_tickets)\
ORDER BY avg_tour_price DESC\
LIMIT 10;
               
                    
'''))

In [13]:
query = (Clients
         .select(Clients.name, Clients.surname,
                 Reviews.text.alias("review_text"),
                 Reviews.date.alias("review_date"),
                 Tours.name.alias("tour_name"),
                 fn.AVG(TourTickets.price).alias("avg_tour_price"))
         .join(Reviews)
         .join(Tours)
         .join(TourTickets)
         .group_by(Clients.id)
         .having(SQL("avg_tour_price") > (TourTickets.select(fn.AVG(TourTickets.price))))
         .order_by(SQL("avg_tour_price").desc())
         .limit(10)
        )

l = [[i.name + " " + i.surname, f"'{i.review_text}'", i.review_date, i.tour_name, i.avg_tour_price] for i in query.namedtuples()]

list(cursor.execute('''\
SELECT \
    c.name || ' ' || c.surname AS candidate_name,\
    p.name AS position_name,\
    c.mark AS candidate_mark,\
    AVG(c2.mark) AS avg_position_mark,\
    COUNT(e.id) AS current_employees_in_position\
FROM candidats c\
JOIN positions p ON c.position_id = p.id\
LEFT JOIN employees e ON e.position_id = p.id\
JOIN candidats c2 ON c2.position_id = p.id\
GROUP BY c.id\
HAVING c.mark > (SELECT AVG(mark) FROM candidats WHERE position_id = p.id)\
ORDER BY c.mark DESC;\
               
                    
'''))

In [14]:
Candidat2 = Candidats.alias()

avg_mark_subq = (Candidat2
                .select(fn.AVG(Candidat2.mark))
                .where(Candidat2.position_id == Candidats.position_id))

query = (Candidats
        .select(
            Candidats.name.concat(' ').concat(Candidats.surname).alias('candidate_name'),
            Positions.name.alias('position_name'),
            Candidats.mark.alias('candidate_mark'),
            fn.AVG(Candidat2.mark).alias('avg_position_mark'),
            fn.COUNT(Employees.id).alias('current_employees_in_position')
        )
        .join(Positions, on=(Candidats.position_id == Positions.id))
        .join(Employees, JOIN.LEFT_OUTER, on=(Employees.position_id == Positions.id))
        .join(Candidat2, on=(Candidat2.position_id == Positions.id))
        .group_by(Candidats.id, Positions.name)
        .having(Candidats.mark > avg_mark_subq)
        .order_by(Candidats.mark.desc()))


for i in query.namedtuples():
    print(i.candidate_name + " " + i.position_name + " " + str(i.candidate_mark) + " " + str(i.avg_position_mark) + " " + str(i.current_employees_in_position))


James Wilson Co-Pilot 10 9.0 6
Lisa Wilson Co-Pilot 10 9.0 6
Robert Wilson Tour Guide 5 4.5 6


list(cursor.execute('''\
SELECT \
    s.name AS spaceship_name,\
    SUM(p.price * r.count) AS total_maintenance_cost\
FROM spaceships s\
JOIN maintain_tickets mt ON s.id = mt.spaceship_id\
JOIN reasons r ON mt.reason_id = r.id\
JOIN products p ON r.product_id = p.id\
GROUP BY s.id\
ORDER BY total_maintenance_cost DESC;\
               
                    
'''))

In [9]:
subquery = fn.SUM(Products.price * Reasons.count)

query = (Spaceships
         .select(
                 Spaceships.name.alias("spaceship_name"),
                 subquery.alias("total_maintenance_cost")
                 )
         .join(MaintainTickets, on=(Spaceships.id == MaintainTickets.spaceship_id))
         .join(Reasons, on=(MaintainTickets.reason_id == Reasons.id))
         .join(Products, on=(Reasons.product_id == Products.id))
         .group_by(Spaceships.id)
         .order_by(subquery.desc())
         
        )

for i in query:
    print(i.spaceship_name, i.total_maintenance_cost)

Galactic Cruiser 33000
Orbital Shuttle 9000
Cosmic Explorer 8000


list(cursor.execute('''\
SELECT \
    a.name AS airport_name,\
    COUNT(t.id) AS tour_count,\
    GROUP_CONCAT(DISTINCT so.name) AS space_objects,\
    SUM(s.human_places) AS total_passenger_capacity\
FROM airports a\
JOIN tours t ON a.id = t.airport_id\
JOIN space_objects so ON t.obj_id = so.id\
JOIN spaceships s ON t.spaceship_id = s.id\
GROUP BY a.id\
ORDER BY tour_count DESC;\
                    
'''))

In [10]:
tour_count = fn.COUNT(Tours.id)
query = (
         Airports
         .select(
             Airports.name.alias("airport_name"),
             tour_count.alias("tour_count"),
             fn.GROUP_CONCAT(SpaceObjects.name.distinct()).alias("space_objects"),
             fn.SUM(Spaceships.human_places).alias("total_passenger_capacity")
         )
         .join(Tours)
         .join(SpaceObjects)
         .switch(Tours)
         .join(Spaceships)
         .group_by(Airports.id)
         .order_by(tour_count.desc())
        )

for i in query:
    print(i.airport_name, i.tour_count, i.space_objects, i.total_passenger_capacity)

Orbital One 3 International Space Station,Moon 155
Galactic Central 3 Venus Orbit,Jupiter Flyby,International Space Station 90
Cosmodrome Beta 2 Venus Orbit 150
Starport Alpha 2 Moon,Mars 80
